# 1.0. FUNCTIONS & LIBS

## 1.1. Imports

In [1]:
import numpy      as np
import pandas     as pd
import seaborn    as sns
import umap.umap_ as umap

from datetime   import datetime
from datetime   import timedelta

from plotly        import express       as px
from sklearn       import cluster       as C
from sklearn       import metrics       as M
from sklearn       import mixture       as mx
from sklearn       import manifold      as mn
from sklearn       import ensemble      as en
from sklearn       import decomposition as dc
from sklearn       import preprocessing as pp
from matplotlib    import pyplot        as plt
from matplotlib    import colormaps     as cm
from scipy.cluster import hierarchy     as hc

# 2.0. Load Data

In [2]:
path = '/home/felipe/repos/insiders_clustering/'

df_raw = pd.read_csv(path + 'data/raw/Ecommerce.csv', encoding='iso-8859-1')
df_raw = df_raw.drop(columns = 'Unnamed: 8')

## 2.3. Rename Columns

In [3]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
df_raw.columns = cols_new

### 2.5.1. Replace NA

In [4]:
# separate rows with missing costumers
df_raw_missing = df_raw.loc[df_raw['customer_id'].isna(), :]

In [5]:
# create reference
df_backup = pd.DataFrame(df_raw_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, (19000+len(df_backup)), 1)

# merge original with reference data
df_raw = pd.merge(df_raw, df_backup, on='invoice_no', how='left')

# coalesce
df_raw['customer_id'] = df_raw['customer_id_x'].combine_first(df_raw['customer_id_y'])

# drop extra columns
df_raw = df_raw.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

## 2.6. Check DTypes

### 2.6.1 Change DTypes

In [6]:
df_raw = df_raw.astype({'customer_id': 'int64'})
df_raw['invoice_date'] = pd.to_datetime(df_raw['invoice_date'], format='%d-%b-%y')

# 3.0. DATA FILTERING

In [7]:
df_raw_3 = df_raw.copy()

In [8]:
# list created using libreoffice calc
exclude_stock_codes = [
'AMAZONFEE',
'B',
'BANK CHARGES',
'C2',
'CRUK',
'D',
'DCGS0003',
'DCGS0004',
'DCGS0055',
'DCGS0057',
'DCGS0066P',
'DCGS0067',
'DCGS0068',
'DCGS0069',
'DCGS0070',
'DCGS0071',
'DCGS0072',
'DCGS0073',
'DCGS0074',
'DCGS0076',
'DCGSSBOY',
'DCGSSGIRL',
'DOT',
'gift_0001_10',
'gift_0001_20',
'gift_0001_30',
'gift_0001_40',
'gift_0001_50',
'M',
'm',
'PADS',
'POST',
'S']

## 3.2. Data Filtering

In [9]:
# unit price
df_raw_3 = df_raw_3.loc[df_raw_3['unit_price']>0.01,:]

# stock code
df_raw_3 = df_raw_3[~df_raw_3['stock_code'].isin(exclude_stock_codes)]

# quantity
df_raw_returns = df_raw_3.loc[df_raw_3['quantity']<0, :]
df_raw_purchases = df_raw_3.loc[df_raw_3['quantity']>=0, :]

# 4.0. FEATURE ENGINEERING

In [10]:
df_raw_4 = df_raw_3.copy()

## 4.1. Features Creation

In [11]:
# separate unique customers
df_ref = df_raw_4.drop(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'country'], axis=1).drop_duplicates(ignore_index=True)

### 4.1.1. Gross Revenue - total money spent by customer

In [12]:
# calculated from total invoices (returns are subtracted from gross revenue)
df_raw_4['gross_revenue'] = df_raw_4['quantity']*df_raw_4['unit_price']
df_gross_revenue = df_raw_4[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_gross_revenue, on='customer_id', how='left')

### 4.1.3. Recency - number of days since last purchase

In [13]:
# recency - last day of purchase
# calculated from dataframe with only purchases
df_recency = df_raw_purchases.loc[:, ['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency'] = (df_raw_4['invoice_date'].max()-df_recency['invoice_date']).dt.days+1
df_recency = df_recency[['customer_id', 'recency']]
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

### 4.1.4. Frequency - number of purchases in last 365 days

In [14]:
# frequency - number of purchases in last 365 days
dd = timedelta(days=365)
df_frequency = df_raw_purchases[df_raw_purchases['invoice_date']>=(df_raw_purchases['invoice_date'].max() - dd)][['invoice_no', 'customer_id']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns={'invoice_no':'frequency'})
df_ref = pd.merge(df_ref, df_frequency, on='customer_id', how='left')

### 4.1.5. Quantity of Returns - number of invoices with negative quantity

In [15]:
df_returns = (df_raw_returns[['customer_id', 'invoice_no']].drop_duplicates()
                                                           .groupby('customer_id')
                                                           .count()
                                                           .reset_index()
                                                           .rename(columns={'invoice_no':'qty_returns'}))
df_ref = pd.merge(df_ref, df_returns, on='customer_id', how='left')

### 4.1.7. Quantity of Products

In [16]:
df_quantity = (df_raw_4[['customer_id','quantity']].groupby('customer_id')
                                                           .sum()
                                                           .reset_index()
                                                           .rename(columns={'quantity': 'qty_items'}))
df_ref = pd.merge(df_ref, df_quantity, on='customer_id', how='left')

### 4.1.9. Assortment- quantity of distinct items purchased

In [17]:
df_assortment = (df_raw_purchases[['customer_id', 'stock_code']].drop_duplicates()
                                                                .groupby('customer_id')
                                                                .count()
                                                                .reset_index()
                                                                .rename(columns={'stock_code': 'assortment'}))
df_ref = pd.merge(df_ref, df_assortment, on='customer_id', how='left')

## 4.2. Check NA and DTypes after Feature Creation

### 4.2.1. Replace NA in df_ref

In [18]:
# nan values were assigned to customers with 0 purchases in the last 365 days
df_ref['frequency'] = df_ref['frequency'].fillna(0)

# nan values were assigned to customers wih 0 returns
df_ref['qty_returns'] = df_ref['qty_returns'].fillna(0)

# drop NA for remaining attributes
df_ref = df_ref.dropna()

In [19]:
# change dtypes
df_ref = df_ref.astype({'recency':'int64', 
                        'frequency': 'int64', 
                        'qty_returns':'int64',
                        'qty_items': 'int64',
                        'assortment':'int64'})

# 5.0. EDA (EXPLORATORY DATA ANALYSIS)

In [20]:
df_ref_5 = df_ref.copy()

## 5.3. Space Analysis

In [21]:
# selected features for analysis
cols_selected = ['gross_revenue', 'recency', 'frequency', 'qty_returns', 'qty_items', 'assortment']
df_ref_53 = df_ref_5[cols_selected]

In [22]:
mm = pp.MinMaxScaler()

df_ref_53['gross_revenue']   = mm.fit_transform(df_ref_53[['gross_revenue']])
df_ref_53['recency']         = mm.fit_transform(df_ref_53[['recency']])
df_ref_53['frequency']       = mm.fit_transform(df_ref_53[['frequency']])
df_ref_53['qty_returns']     = mm.fit_transform(df_ref_53[['qty_returns']])
df_ref_53['qty_items']       = mm.fit_transform(df_ref_53[['qty_items']])
df_ref_53['assortment']      = mm.fit_transform(df_ref_53[['assortment']])

/tmp/ipykernel_10773/3838092104.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ref_53['gross_revenue']   = mm.fit_transform(df_ref_53[['gross_revenue']])
/tmp/ipykernel_10773/3838092104.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ref_53['recency']         = mm.fit_transform(df_ref_53[['recency']])
/tmp/ipykernel_10773/3838092104.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

### 5.3.4. Tree-Based Embedding

In [23]:
# training dataset
X = df_ref_5[['recency', 'frequency', 'qty_returns', 'qty_items', 'assortment']]
y = df_ref_5['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor(n_estimators=100, random_state=42)

# model training
rf_model.fit(X, y)

# dataframe leafs
df_leaf = pd.DataFrame(rf_model.apply(X))

In [24]:
# reducer of dimensionality
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

# 8.0. HYPER-PARAMETER FINE TUNING

In [25]:
X = df_tree.copy()
X.to_csv(path + 'src/data/tree_based_embedding.csv', index=False)

# 9.0. MODEL TRAINING

## 9.1. K-Means

In [26]:
# model definition
k = 8
kmeans = C.KMeans(init='random', n_clusters=k, n_init=100, max_iter=300, random_state=42)

# model training
kmeans.fit(X)

# clustering
labels = kmeans.labels_

### 9.1.2. K-Means cluster validation

In [27]:
# WSS (Within-Cluster Sum of Square)
print('WSS value: {}'.format(kmeans.inertia_))

# SS (Silhouette Score)
print('SS value: {}'.format(M.silhouette_score(X, labels, metric='euclidean')))

WSS value: 88500.6328125


SS value: 0.49807924032211304


## 10.2. Clusters Profile

In [28]:
df_ref_10 = df_ref_5[['customer_id',
                      'gross_revenue',
                      'recency',
                      'frequency',
                      'qty_returns',
                      'qty_items',
                      'assortment']].copy()
df_ref_10['cluster'] = labels
df_ref_10['last_training_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [29]:
# number of customer
df_cluster = df_ref_10[['customer_id', 'cluster']].groupby('cluster').count().reset_index().rename(columns={'customer_id':'qty_customers'})
df_cluster['perc_customer'] = 100*(df_cluster['qty_customers']/df_cluster['qty_customers'].sum())

# avg gross revenue
df_avg_gr = df_ref_10[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index().rename(columns={'gross_revenue':'avg_gross_revenue'})
df_cluster = pd.merge(df_cluster, df_avg_gr, on='cluster', how='left')

# avg recency days
df_avg_rd = df_ref_10[['recency', 'cluster']].groupby('cluster').mean().reset_index().rename(columns={'recency':'avg_recency'})
df_cluster = pd.merge(df_cluster, df_avg_rd, on='cluster', how='left')

# avg frequency
df_avg_fr = df_ref_10[['frequency', 'cluster']].groupby('cluster').mean().reset_index().rename(columns={'frequency':'avg_frequency'})
df_cluster = pd.merge(df_cluster, df_avg_fr, on='cluster', how='left')

# avg_qty_returns
df_avg_qr = df_ref_10[['qty_returns', 'cluster']].groupby('cluster').mean().reset_index().rename(columns={'qty_returns':'avg_qty_returns'})
df_cluster = pd.merge(df_cluster, df_avg_qr, on='cluster', how='left')

# avg_qty_items
df_avg_qi = df_ref_10[['qty_items', 'cluster']].groupby('cluster').mean().reset_index().rename(columns={'qty_items':'avg_qty_items'})
df_cluster = pd.merge(df_cluster, df_avg_qi, on='cluster', how='left')

# avg_assortment
df_avg_as = df_ref_10[['assortment', 'cluster']].groupby('cluster').mean().reset_index().rename(columns={'assortment':'avg_assortment'})
df_cluster = pd.merge(df_cluster, df_avg_as, on='cluster', how='left')


In [30]:
df_cluster.head()

,cluster,qty_customers,perc_customer,avg_gross_revenue,avg_recency,avg_frequency,avg_qty_returns,avg_qty_items,avg_assortment
0,0,448,7.852761,6298.593482,32.872768,11.799107,2.636161,3779.437500,167.772321
1,1,926,16.231376,978.147808,106.653348,2.230022,0.382289,447.879050,66.853132
2,2,1100,19.281332,293.810791,166.717273,1.137273,0.136364,118.072727,24.537273
3,3,554,9.710780,4402.583935,180.252708,4.001805,0.586643,2515.128159,35.978339
4,4,484,8.483786,2786.180950,56.933884,5.533058,1.074380,1525.828512,160.276860


### 10.2.1. Cluster Insiders

- Cluster: 0
- Number of customers: 448 (7.85% of customer base)
- Average recency (days since last purchase): 33 days
- Average frequency (purchases in last 365 days): 12 purchases
- Average gross revenue: $ 6,299.00
- Average quantity of returns: 2.6
- Average quantity of items purchased: 3779
- Average assortment: 168

# 12.0. DEPLOY

In [31]:
df_ref_10.dtypes

customer_id                  int64
gross_revenue              float64
recency                      int64
frequency                    int64
qty_returns                  int64
qty_items                    int64
assortment                   int64
cluster                      int32
last_training_timestamp     object
dtype: object

In [32]:
df_ref_10.head()

,customer_id,gross_revenue,recency,frequency,qty_returns,qty_items,assortment,cluster,last_training_timestamp
0,17850,5288.63,373,0,1,1693,21,4,2022-12-09 20:53:32
1,13047,3089.10,57,6,7,1355,105,4,2022-12-09 20:53:32
2,12583,6629.34,3,14,2,4978,114,0,2022-12-09 20:53:32
3,13748,948.25,96,4,0,439,24,1,2022-12-09 20:53:32
4,15100,635.10,334,1,3,58,1,3,2022-12-09 20:53:32


## 12.1. Insert Data Into SQLite

In [33]:
# # create table
# query_create_table = """
#     CREATE TABLE insiders (
#         customer_id             INTEGER,
#         gross_revenue           REAL,
#         recency                 INTEGER,
#         frequency               INTEGER,
#         qty_returns             INTEGER,
#         qty_items               INTEGER,
#         assortment              INTEGER,
#         cluster                 INTEGER,
#         last_training_timestamp TEXT
#     )
# """

# conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute(query_create_table)
# conn.commit()
# conn.close()

In [34]:
# # drop table
# query_drop_table = """
#     DROP TABLE insiders
# """

# conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute(query_drop_table)
# conn.commit()
# conn.close()

In [35]:
# # insert data
# conn = create_engine('sqlite:///insiders_db.sqlite')
# df_ref_10.to_sql('insiders', con=conn, if_exists='append', index=False)

In [36]:
# # consulting database
# query = """
#     SELECT * FROM insiders
# """

# df = pd.read_sql(query, conn)